In [50]:
import httpx
from decouple import config
from decouple import Config, RepositoryEnv, config
import pathlib

NBS_DIR = pathlib.Path().resolve()
REPO_DIR = NBS_DIR.parent

config_env = Config(RepositoryEnv(REPO_DIR / '.env'))

API_ACCESS_KEY = config_env('API_ACCESS_KEY')

headers = {
    "X-API-Key" : API_ACCESS_KEY
}
endpoint = "http://127.0.0.1:8000/"
# prompt = "a phot of TOK who is a architecture explaining building plans to a contractor"
res = httpx.get(endpoint, headers=headers)
res.status_code, res.json(), res.headers.get('Retry-After'), type(res.status_code)

(200, {'Hello': 'World'}, None, int)

In [53]:
API_ACCESS_KEY = config_env('API_ACCESS_KEY')

headers = {
    "X-API-Key" : API_ACCESS_KEY
}
endpoint = "http://127.0.0.1:8000/predictions"
endpoint = "http://127.0.0.1:8000/processing"
# prompt = "a phot of TOK who is a architecture explaining building plans to a contractor"
# res = httpx.get(endpoint, params={"status":"processing"}, headers=headers)
# res = httpx.get(endpoint, params={"status":"succeeded"}, headers=headers)
res = httpx.get(endpoint, headers=headers)
preds_json = res.json()
res.status_code, preds_json, res.headers.get('Retry-After')

(200,
 [{'id': '8j6ec6hkk9rme0cpa78bh14vbm',
   'url': '/predictions/8j6ec6hkk9rme0cpa78bh14vbm',
   'status': 'processing',
   'created_at': '2025-04-20T06:08:41.370000Z',
   'started_at': '2025-04-20T06:08:41.389645Z',
   'completed_at': None}],
 None)

In [49]:
preds_json
res.json()

[{'id': '1b5xwge49nrm80cpa5j8bd8zwc',
  'url': '/predictions/1b5xwge49nrm80cpa5j8bd8zwc',
  'model': 'slackroo/superme-sumanth-v1',
  'version': '535caef9907be746cc55ccde50aa5eee85d8f987d11c8c4735d70a18d1e37f49',
  'status': 'succeeded',
  'created_at': '2025-04-20T04:11:20.525000Z',
  'completed_at': '2025-04-20T04:11:49.850134Z',
  'files': ['/predictions/1b5xwge49nrm80cpa5j8bd8zwc/files/0.jpg',
   '/predictions/1b5xwge49nrm80cpa5j8bd8zwc/files/1.jpg'],
  'num_outputs': 2},
 {'id': '66habag1bhrmc0cpa4asnc89jr',
  'url': '/predictions/66habag1bhrmc0cpa4asnc89jr',
  'model': 'slackroo/superme-sumanth-v1',
  'version': '535caef9907be746cc55ccde50aa5eee85d8f987d11c8c4735d70a18d1e37f49',
  'status': 'succeeded',
  'created_at': '2025-04-20T02:44:13.276000Z',
  'completed_at': '2025-04-20T02:44:49.869277Z',
  'files': [],
  'num_outputs': 0},
 {'id': 'rxcqyjedvhrme0cp95nbtv592c',
  'url': '/predictions/rxcqyjedvhrme0cp95nbtv592c',
  'model': 'slackroo/superme-sumanth-v1',
  'version': '535

In [33]:
# detailed data
API_ACCESS_KEY = config_env('API_ACCESS_KEY')

headers = {
    "X-API-Key" : API_ACCESS_KEY
}
# endpoint = "http://127.0.0.1:8000/predictions"
recent_url = res.json()[0].get('url')
if not recent_url.startswith('/'):
    recent_url = f"/{recent_url}"
detail_url = f"http://127.0.0.1:8000{recent_url}"
# detail_url = f"http://127.0.0.1:8000/predictions/asdf"
# prompt = "a phot of TOK who is a architecture explaining building plans to a contractor"
preds_res = httpx.get(detail_url, headers=headers)
preds_res.status_code
preds_res.json()

{'id': '66habag1bhrmc0cpa4asnc89jr',
 'model': 'slackroo/superme-sumanth-v1',
 'version': '535caef9907be746cc55ccde50aa5eee85d8f987d11c8c4735d70a18d1e37f49',
 'status': 'succeeded',
 'input': {'aspect_ratio': '1:1',
  'extra_lora_scale': 1,
  'go_fast': False,
  'guidance_scale': 3,
  'image': 'https://api.replicate.com/v1/files/ZDkyY2ExODktZGMwNS00OGJhLThkYjQtM2JjM2E3ZjExZjJm',
  'lora_scale': 1,
  'megapixels': '1',
  'model': 'dev',
  'num_inference_steps': 28,
  'num_outputs': 4,
  'output_format': 'jpg',
  'output_quality': 80,
  'prompt': 'a phot of TOK adult man dressed up for a professional photoshoot.',
  'prompt_strength': 0.8,
  'replicate_weights': 'https://replicate.delivery/xezq/8fHrvMHfZGueCJpperFX7mI6fHKRkS6Hodl5WDowUuBeuEVIF/trained_model.tar'},
 'output': ['https://replicate.delivery/xezq/xsL5FEanxVomNRmO6PMJR0TutebNluAzxDog8tLOiNiQjQSKA/out-0.jpg',
  'https://replicate.delivery/xezq/0RRScErbo0L4GhtGk4X52kEDXFBX67XhynUS6e3NfUVhGhkUA/out-1.jpg',
  'https://replicate.de

In [39]:
for obj in preds_json:
    path = obj.get('url')
    endpoint = f"http://127.0.0.1:8000{path}"
    res = httpx.get(endpoint, headers=headers)
    if res.status_code not in range(200, 299):
        continue
    data = res.json()
    # print(data)
    _id = data.get('id')
    _input = data.get('input') or {}
    num_outputs = _input.get('num_outputs') or 0
    _output = data.get('output')
    if _output is None:
        continue
    output_names = [pathlib.Path(x) for x in _output]
    print(_id, _input, _output)
    print(output_names)
    for x,output_path in enumerate(output_names):
        suffix = output_path.suffix
        print(f"{path}/file/{x}{suffix}")

    print(data)
    break



66habag1bhrmc0cpa4asnc89jr {'aspect_ratio': '1:1', 'extra_lora_scale': 1, 'go_fast': False, 'guidance_scale': 3, 'image': 'https://api.replicate.com/v1/files/ZDkyY2ExODktZGMwNS00OGJhLThkYjQtM2JjM2E3ZjExZjJm', 'lora_scale': 1, 'megapixels': '1', 'model': 'dev', 'num_inference_steps': 28, 'num_outputs': 4, 'output_format': 'jpg', 'output_quality': 80, 'prompt': 'a phot of TOK adult man dressed up for a professional photoshoot.', 'prompt_strength': 0.8, 'replicate_weights': 'https://replicate.delivery/xezq/8fHrvMHfZGueCJpperFX7mI6fHKRkS6Hodl5WDowUuBeuEVIF/trained_model.tar'} ['https://replicate.delivery/xezq/xsL5FEanxVomNRmO6PMJR0TutebNluAzxDog8tLOiNiQjQSKA/out-0.jpg', 'https://replicate.delivery/xezq/0RRScErbo0L4GhtGk4X52kEDXFBX67XhynUS6e3NfUVhGhkUA/out-1.jpg', 'https://replicate.delivery/xezq/j6MJlDiKhg6OCFKr0xtWiVWU5bofDGgRmqHNr0VSmavQjQSKA/out-2.jpg', 'https://replicate.delivery/xezq/EfIlZnNEgH1IJyxWhZOEtvtmWAbB8fo3qNmzWm9jVBYhGhkUA/out-3.jpg']
[PosixPath('https:/replicate.delivery/xe

In [7]:
res.status_code == httpx.codes.OK


True